In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!pip install tqdm transformers sacrebleu sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.2 MB/s eta 0:00:00


# Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [ ]:
df_train_transcript = pd.read_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/ALIGNED/MERGE/TRAIN_ALL.csv')
df_train_transcript = df_train_transcript.to_dict(orient='dict')

df_hanja_transcript = pd.read_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/KO_HANJA_DICT_ALL_V2.csv')
df_hanja_transcript = df_hanja_transcript.to_dict(orient='dict')

TRAIN_ESENT = (list(df_train_transcript['EN'].values()))
TRAIN_KSENT = (list(df_train_transcript['KO'].values())) #+ (list(df_hanja_transcript['WORD'].values()))
TRAIN_ZSENT = (list(df_train_transcript['ZH-TW'].values())) #+ (list(df_hanja_transcript['HANJA'].values()))
df_train_transcript = []

In [ ]:
df_test_transcript = pd.read_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/ALIGNED/MERGE/TEST_ALL.csv')
df_test_transcript = df_test_transcript.to_dict(orient='dict')
TEST_ESENT = (list(df_test_transcript['EN'].values()))
TEST_KSENT = (list(df_test_transcript['KO'].values()))
TEST_ZSENT = (list(df_test_transcript['ZH-TW'].values()))
df_test_transcript = []

In [3]:
df_tatoeba = pd.read_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/Sentence pairs in Korean-Mandarin Chinese - 2024-03-25.tsv',sep='\t',names=['id_ko','ko','id_zh','zh'])
df_tatoeba.tail()

,id_ko,ko,id_zh,zh
537,11074151,맞아.,12278435,對的。
538,4710369,서둘러!,12278451,快點啦！
539,6485249,저기 봐.,12278503,看那裏！
540,12301479,나는 의사예요.,2370671,我是医生。
541,4717732,현재 다토에바에는 몽골어 문장이 여섯 개다.,683039,Tatoeba 現在有六句蒙古語的句子。


In [4]:
df_tatoeba = df_tatoeba.to_dict(orient='dict')
print(len((list(df_tatoeba['ko'].values()))))

542


In [5]:
TRAIN_KSENT = (list(df_tatoeba['ko'].values()))[:433]
TRAIN_ZSENT = (list(df_tatoeba['zh'].values()))[:433]
TEST_KSENT = (list(df_tatoeba['ko'].values()))[433:]
TEST_ZSENT = (list(df_tatoeba['zh'].values()))[433:]


# Preprocess

In [6]:
from transformers import AutoTokenizer
#model_checkpoint = 'Helsinki-NLP/opus-mt-en-ro'
#model_checkpoint = 'google/mt5-base'
model_checkpoint = 'facebook/mbart-large-cc25'
#model_checkpoint = '/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/mt5_1/checkpoint-8000'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True) # 用fast版本的tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [15]:
tokenizer.src_lang = 'zh_CN'
tokenizer.tgt_lang = 'ko_KR'

In [19]:
model_input = tokenizer(TRAIN_ZSENT[0])
tokens = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
print(model_input)
print('tokens: {}'.format(tokens))

with tokenizer.as_target_tokenizer():
  model_input = tokenizer(TRAIN_KSENT[0])
  print(model_input)
  tokens = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
  print('tokens: {}'.format(tokens))

{'input_ids': [61317, 176249, 2112, 38, 2, 250025], 'attention_mask': [1, 1, 1, 1, 1, 1]}
tokens: ['▁我們', '試試', '看', '!', '</s>', 'zh_CN']
{'input_ids': [208798, 8280, 125199, 38, 2, 250014], 'attention_mask': [1, 1, 1, 1, 1, 1]}
tokens: ['▁뭔가', '▁해', '보자', '!', '</s>', 'ko_KR']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
print(tokenizer.convert_ids_to_tokens(250014))
print(tokenizer.convert_ids_to_tokens(250025))

ko_KR
zh_CN


In [20]:
max_input_length = 16
max_target_length = 16
source_lang = 'ZH-TW'
target_lang = 'KO'
prefix = ''#'translate Chinese to Korean: '

In [21]:
def preprocess_function(source_sentence, target_sentence):
  global tokenizer, max_input_length, max_target_length, prefix, source_lang, target_lang
  inputs = prefix + source_sentence
  targets = target_sentence
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

  #Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

  model_inputs['labels'] = labels['input_ids']
  #model_inputs['translation'] = {source_lang:source_sentence,target_lang:target_sentence}
  #print(model_inputs.keys())
  #for key in model_inputs.keys():
  #  print(key,':',model_inputs[key])
  return model_inputs

In [22]:
from tqdm import tqdm
def map_like_datasets(source_sentences, target_sentences):
  global source_lang, target_lang
  my_datasets = []
  with tqdm(total = len(source_sentences)) as pbar:
   for i in range(len(source_sentences)):
    my_datasets.append(preprocess_function(source_sentences[i],target_sentences[i]))
    pbar.update(1)
  return my_datasets

In [23]:
train_tokenized_datasets = map_like_datasets(TRAIN_ZSENT,TRAIN_KSENT)
test_tokenized_datasets = map_like_datasets(TEST_ZSENT,TEST_KSENT)

100%|██████████| 109/109 [00:00<00:00, 2204.80it/s]


In [24]:
print(train_tokenized_datasets[:5])
print(test_tokenized_datasets[:5])

[{'input_ids': [61317, 176249, 2112, 38, 2, 250025, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [208798, 8280, 125199, 38, 2, 250014, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [13129, 4941, 1677, 134753, 274, 30, 2, 250025, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [8511, 6838, 17937, 5, 2, 250014, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [6, 97092, 9811, 17194, 6147, 32, 2, 250025, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [82790, 1963, 32, 2, 250014, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [46246, 115424, 6147, 32, 2, 250025, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [126061, 8143, 172651, 13187, 48331, 32, 2, 250014, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [61168, 354, 910, 630, 1819, 5730, 4, 4264, 10685, 14, 15905

In [ ]:
print(len(train_tokenized_datasets[0]['labels']))

64


# Train

In [26]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

In [27]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/mt5_tatoeba_mbart",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    #load_best_model_at_end=True,
    #logging_steps=8000,
    #save_steps=2000
)

# If you have error here, just restart and rerun all cells without '!pip install xxx'.

In [30]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


# Evaluation

In [29]:
import numpy as np
from datasets import load_metric
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-29-3dcc54f82161>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from sacrebleu.metrics import BLEU

def sacrebleu_score(source_sentences,target_sentences,lang='ko'):
  global my_tokenizer, model, max_length, device
  model.eval()
  # 生成目標語句
  generated_sentences = []
  with tqdm(total = len(source_sentences)) as pbar:
    for source_sentence in source_sentences:
      source_tokens = my_tokenizer(source_sentence, padding=True, truncation=True, return_tensors='pt', max_length=max_length).to(device)
      with torch.no_grad():
        output = model(input_ids=source_tokens.input_ids, attention_mask=source_tokens.attention_mask)
      generated_sentence = my_tokenizer.decode(output[0].argmax(dim=-1), skip_special_tokens=True)
      generated_sentences.append(generated_sentence)
      pbar.update(1)

  bleu = BLEU()
  bleu.trg_lang = lang
  bleu_score = bleu.corpus_score(generated_sentences,[[sent] for sent in target_sentences])
  #print('BLEU Score:',bleu.score)
  print(bleu_score)
  return bleu_score.score

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)